In [ ]:
!pip install pandas-ta

In [ ]:
#Librerías a utilizar
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go


In [ ]:
#Definir Ticker
Ticker='NIO'
#Definir la comision de Compra / Venta
Comision=0.0025 #Comisión de 0.25% por compra o venta de casa de bolsa

In [ ]:
#Definición de funciones a utilizar

#Función de extracción de información de Yahoo Finance de 1 mes cada 60 minutos
def get_data(symbol: str):
    data=yf.download(tickers=symbol, period='44d', interval='60m')
    data.reset_index(inplace=True)
    return data

# Función para calcular el Promedio Móvil
def calculate_sma(data, length: int):
    return ta.sma(data['Close'], length)

# Función para determinar la tendencia - equilibrada
def determine_trend(data):
    if data['SMA_10']>data['SMA_20']>data['SMA_30']:
        return 2 # Tendencia de Alza
    elif data['SMA_10']<data['SMA_20']<data['SMA_30']:
        return 1 # Tendencia de Baja
    else:
        return 0 # Sin tendencia definida


# Función para calcular RSI (Relative Strength Index)
# Se utliza en los mercados como un indicar de Sobrecompra o sobreventa
# Valores por arriba de 70% indican que estan en un territorio de sobrecompra
# Valores por abajo de 30% indican que estan en un territorio de sobreventa
def rsi(DF, n=20):
    df=DF.copy()
    df['dif'] = df['Close']-df['Close'].shift(1)
    df['gain'] = np.where(df['dif']>=0, df['dif'], 0)
    df['loss'] = np.where(df['dif']<0, abs(df['dif']), 0)
    average_gain = []
    average_loss = []
    gain=df['gain'].tolist()
    loss=df['loss'].tolist()
    for i in range(len(df)):
      if i < n:
        average_gain.append(np.NaN)
        average_loss.append(np.NaN)
      elif i == n:
        average_gain.append(df['gain'].rolling(n).mean()[n])
        average_loss.append(df['loss'].rolling(n).mean()[n])
      elif i > n:
        average_gain.append(((n-1)*average_gain[i-1]+gain[i])/n)
        average_loss.append(((n-1)*average_loss[i-1]+loss[i])/n)
    df['average_gain'] = np.array(average_gain)
    df['average_loss'] = np.array(average_loss)
    df['RS'] = df['average_gain']/df['average_loss']
    df['RSI'] = 100-(100/(1+df['RS']))
    return df['RSI']




In [ ]:
#Obtener la información de Yahoo Finance y renombrar columnas
data=get_data(Ticker)
data=data.set_axis(['Datetime','Close','High','Low','Open','Volume'], axis=1)

#Calcular los promedios moviles para los ultimos 10, 20 y 30  valores de la columa 'Close' y se agregan al data frame
data['SMA_10']=calculate_sma(data,10)
data['SMA_20']=calculate_sma(data,20)
data['SMA_30']=calculate_sma(data,30)

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Determinar la tendencia en una nueva columna del dataframe
data['Tendencia']=data.apply(determine_trend,axis=1)

In [ ]:
# Se calcula el RSI y se integra al Dataframe
RSI_data=rsi(data,n=20)
data=pd.concat([data,RSI_data], axis=1)
data

,Datetime,Close,High,Low,Open,Volume,SMA_10,SMA_20,SMA_30,Tendencia,RSI
0,2025-01-08 14:30:00+00:00,4.3410,4.4050,4.26,4.4000,20698554,NaN,NaN,NaN,0,NaN
1,2025-01-08 15:30:00+00:00,4.3350,4.3600,4.33,4.3450,4716367,NaN,NaN,NaN,0,NaN
2,2025-01-08 16:30:00+00:00,4.2899,4.3600,4.28,4.3400,5619020,NaN,NaN,NaN,0,NaN
3,2025-01-08 17:30:00+00:00,4.3279,4.3300,4.27,4.2850,4447901,NaN,NaN,NaN,0,NaN
4,2025-01-08 18:30:00+00:00,4.3425,4.3500,4.30,4.3288,3191495,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
303,2025-03-13 15:30:00+00:00,4.7200,4.7900,4.71,4.7750,6373907,5.02112,4.940875,4.791883,2,47.932497
304,2025-03-13 16:30:00+00:00,4.6650,4.7500,4.66,4.7200,6077398,4.96622,4.950130,4.801570,2,45.791988
305,2025-03-13 17:30:00+00:00,4.6995,4.7086,4.65,4.6700,3681310,4.92367,4.962855,4.811720,0,47.344609
306,2025-03-13 18:30:00+00:00,4.7400,4.7700,4.70,4.7000,4859486,4.88306,4.976855,4.821387,0,49.144504


In [ ]:
# Determinar en que periodos hubo tendencia de alza y de baja - se guarda en un nuevo dataframe y lo reindexamos
df=data[data['Tendencia']!=0]
df=df.reset_index()
df

,index,Datetime,Close,High,Low,Open,Volume,SMA_10,SMA_20,SMA_30,Tendencia,RSI
0,29,2025-01-15 15:30:00+00:00,4.0400,4.08,4.03,4.065,3961195,4.07180,4.091760,4.157580,1,27.942204
1,30,2025-01-15 16:30:00+00:00,4.0350,4.04,4.02,4.034,2767405,4.06880,4.083760,4.147380,1,27.626352
2,31,2025-01-15 17:30:00+00:00,4.0650,4.07,4.03,4.030,2418920,4.06955,4.077260,4.138380,1,32.448889
3,32,2025-01-15 18:30:00+00:00,4.1188,4.12,4.06,4.065,2686066,4.07393,4.074450,4.132677,1,39.996459
4,40,2025-01-16 19:30:00+00:00,4.1607,4.19,4.16,4.170,1582888,4.13496,4.101880,4.100827,2,46.642487
...,...,...,...,...,...,...,...,...,...,...,...,...
161,300,2025-03-12 19:30:00+00:00,5.1050,5.12,5.08,5.105,6044944,5.15726,4.912855,4.746383,2,67.134443
162,301,2025-03-13 13:30:00+00:00,4.8050,5.01,4.72,5.000,27123072,5.11462,4.922605,4.765383,2,51.397627
163,302,2025-03-13 14:30:00+00:00,4.7750,4.82,4.74,4.805,8280100,5.06962,4.931195,4.782883,2,50.159968
164,303,2025-03-13 15:30:00+00:00,4.7200,4.79,4.71,4.775,6373907,5.02112,4.940875,4.791883,2,47.932497


In [ ]:
# Determinar la señal de Compra o Venta y definir el precio de dicha señal
df.loc[0,'Señal']=df.loc[0,'Tendencia']
for i in range(1,len(df)):
    if df.loc[i,'Tendencia']==df.loc[i-1,'Tendencia']:
        df.loc[i,'Señal']=0
    else:
        df.loc[i,'Señal']=df.loc[i,'Tendencia']

In [ ]:
#Corregir la primera fila, en caso de que la primera señal, sea una señal de venta
if df.loc[0,'Señal']==1:
    df.loc[0,'Señal']=0
else:
    df.loc[0,'Señal']=2

In [ ]:
#Quitar todos los ceros del dataframe con la columna 'Señal'
df=df[df['Señal']!=0]

#Reindexar finalmente el dataframe
df_Final=df.reset_index()
df_Final

,level_0,index,Datetime,Close,High,Low,Open,Volume,SMA_10,SMA_20,SMA_30,Tendencia,RSI,Señal
0,4,40,2025-01-16 19:30:00+00:00,4.1607,4.19,4.1600,4.1700,1582888,4.13496,4.101880,4.100827,2,46.642487,2.0
1,20,65,2025-01-23 16:30:00+00:00,4.1050,4.14,4.0900,4.1399,6622990,4.20330,4.223340,4.228797,1,40.687876,1.0
2,30,77,2025-01-27 14:30:00+00:00,4.3850,4.42,4.3200,4.3200,13843124,4.25028,4.220010,4.215820,2,64.309926,2.0
3,52,131,2025-02-05 19:30:00+00:00,4.1957,4.23,4.1900,4.2250,5069847,4.29839,4.309945,4.337480,1,42.892161,1.0
4,65,154,2025-02-11 14:30:00+00:00,4.1450,4.21,4.1246,4.2100,18130589,4.28194,4.252875,4.251247,2,40.106985,2.0
5,69,160,2025-02-11 20:30:00+00:00,4.0450,4.08,4.0300,4.0700,14243273,4.16914,4.217370,4.218437,1,34.843550,1.0
6,79,178,2025-02-14 17:30:00+00:00,4.3745,4.40,4.3650,4.3750,2734734,4.30104,4.228060,4.224517,2,60.446156,2.0
7,97,204,2025-02-21 15:30:00+00:00,4.5350,4.58,4.5100,4.5500,17058949,4.39452,4.396245,4.396827,1,65.050933,1.0
8,98,205,2025-02-21 16:30:00+00:00,4.5350,4.57,4.5200,4.5350,16620289,4.40952,4.403000,4.400820,2,65.050933,2.0
9,109,223,2025-02-25 20:30:00+00:00,4.2750,4.31,4.2700,4.2950,5071960,4.30550,4.376505,4.377010,1,42.700749,1.0


In [ ]:
#Calcular el rendimiento de la estrategia
for j in range(0,len(df_Final)):
    if df_Final.loc[j,'Señal']==1:
        df_Final.loc[j,'Rendimiento']=df_Final.loc[j,'Close']*(1-Comision*1.16)/(df_Final.loc[j-1,'Close']*(1+Comision*1.16))-1
    else:
        df_Final.loc[j,'Rendimiento']=0

In [ ]:
#Renombrar la columna 'Señal'
for j in range(0,len(df_Final)):
    if df_Final.loc[j,'Tendencia']==2:
       df_Final.loc[j,'Señal']= "Compra"
    else:
        df_Final.loc[j,'Señal']="Venta"

<ipython-input-313-46ab7321370d>:4: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Compra' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [ ]:
#Generar un listado de columnas para mi resumen del Dataframe
colunmas=['index','Datetime','Close','Señal','RSI','Rendimiento']

#Seleccionar las columnas del dataframe 'df_Final'
df_Resumen=df_Final[colunmas]
df_Resumen


,index,Datetime,Close,Señal,RSI,Rendimiento
0,40,2025-01-16 19:30:00+00:00,4.1607,Compra,46.642487,0.000000
1,65,2025-01-23 16:30:00+00:00,4.1050,Venta,40.687876,-0.019093
2,77,2025-01-27 14:30:00+00:00,4.3850,Compra,64.309926,0.000000
3,131,2025-02-05 19:30:00+00:00,4.1957,Venta,42.892161,-0.048703
4,154,2025-02-11 14:30:00+00:00,4.1450,Compra,40.106985,0.000000
5,160,2025-02-11 20:30:00+00:00,4.0450,Venta,34.843550,-0.029769
6,178,2025-02-14 17:30:00+00:00,4.3745,Compra,60.446156,0.000000
7,204,2025-02-21 15:30:00+00:00,4.5350,Venta,65.050933,0.030695
8,205,2025-02-21 16:30:00+00:00,4.5350,Compra,65.050933,0.000000
9,223,2025-02-25 20:30:00+00:00,4.2750,Venta,42.700749,-0.062783


In [ ]:
df_Stats=df_Resumen[df_Resumen['Rendimiento']!=0]
df_Stats['Rendimiento'].describe().round(4)

,Rendimiento
count,6.0000
mean,-0.0464
std,0.0596
min,-0.1490
25%,-0.0593
50%,-0.0392
75%,-0.0218
max,0.0307


In [ ]:
#Crear una columna que me señale la Compra y a que precio
for j in range (0,len(df_Resumen)):
    if df_Resumen.loc[j,'Señal']=='Compra':
        df_Resumen.loc[j,'Señal_Compra']=df_Resumen.loc[j,'Close']
    else:
        df_Resumen.loc[j,'Señal_Compra']=""

#Crear una columna que me señale la Venta y a que precio
for j in range (0,len(df_Resumen)):
    if df_Resumen.loc[j,'Señal']=='Venta':
        df_Resumen.loc[j,'Señal_Venta']=df_Resumen.loc[j,'Close']
    else:
        df_Resumen.loc[j,'Señal_Venta']=""

<ipython-input-316-d8c5aad748a3>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-316-d8c5aad748a3>:6: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

<ipython-input-316-d8c5aad748a3>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Grafica de velas japonesas y marcadores de compra y venta
grafico=data[:]
fig=go.Figure(data=[go.Candlestick(x=grafico.index,
                                  open=grafico['Open'],
                                  high=grafico['High'],
                                  low=grafico['Low'],
                                  close=grafico['Close'])])
#Añadir los promedios móviles
fig.add_trace(go.Scatter(x=grafico.index,y=grafico['SMA_10'],mode='lines',name='SMA 10', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=grafico.index,y=grafico['SMA_20'],mode='lines',name='SMA 20', line=dict(color='red')))
fig.add_trace(go.Scatter(x=grafico.index,y=grafico['SMA_30'],mode='lines',name='SMA 30', line=dict(color='green')))


#Añadir señales de compra y venta
fig.add_scatter(x=df_Resumen['index'],y=df_Resumen['Señal_Compra'],mode='markers',marker=dict(size=8,color='Black'),name='Compra')
fig.add_scatter(x=df_Resumen['index'],y=df_Resumen['Señal_Venta'],mode='markers',marker=dict(size=8,color='Yellow'),name='Venta')



fig.show()
